In [1]:
!pip install langchain
!pip install boto3
!pip install langchain-community
!pip install langchain-aws
!pip install sentence_transformers
!pip install gpt4all
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 829.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 857.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached n

In [2]:
!python -c "from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings; embedding_model = GPT4AllEmbeddings(model_name='all-MiniLM-L6-v2.gguf2.f16.gguf')"

Downloading: 100% 45.9M/45.9M [00:00<00:00, 74.3MiB/s]
Verifying: 100% 45.9M/45.9M [00:00<00:00, 448MiB/s]
Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcuda.so.1: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcuda.so.1: cannot open shared object file: No such file or directory


In [3]:
# !pip install aws configure
# !pip install awscli

# !aws configure

In [5]:
import boto3
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings
from langchain_aws.llms.bedrock import Bedrock
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [6]:
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def get_bedrock_runtime(region_name, *args, **kwargs):
    return boto3.client(service_name='bedrock-runtime', region_name=region_name, *args, **kwargs)


def get_langchain_bedrock_llm(model_id, client, *args, **kwargs):
    return Bedrock(model_id=model_id, client=client, *args, **kwargs)

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=20,
    length_function=len
)

In [9]:
vectorstoredir = '/content/drive/MyDrive/Apologetics Vectorstore'
# embed_model_name = 'all-MiniLM-L6-v2.gguf2.f16.gguf'
embed_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
llm_name = 'mistral.mixtral-8x7b-instruct-v0:1'

In [15]:
aws_access_token = userdata.get('AWS_STEPHENHO_SERVERLESS_ACC_ACCESS_TOKEN')
aws_secret_key = userdata.get('AWS_STEPHENHO_SERVERLESS_ACC_SECRET_ACCESS_KEY')

config = {
      "max_tokens": 4096,
      "temperature": 0.7,
      "top_p": 0.8
    }

bedrock_runtime = get_bedrock_runtime('us-east-1', aws_access_key_id=aws_access_token, aws_secret_access_key=aws_secret_key)
# bedrock_runtime = get_bedrock_runtime('us-east-1')
# llm = get_langchain_bedrock_llm(llm_name, bedrock_runtime, config=config)
llm = get_langchain_bedrock_llm(llm_name, bedrock_runtime)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Bedrock` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use BedrockLLM instead.
  warn_deprecated(


In [12]:
# embedding_model = GPT4AllEmbeddings(model_name=embed_model_name)
embedding_model = HuggingFaceEmbeddings(model_name=embed_model_name)
if embedding_model.client.tokenizer.pad_token is None:
    embedding_model.client.tokenizer.pad_token = embedding_model.client.tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
db = FAISS.load_local(vectorstoredir, embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

In [16]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever, return_source_documents=True)

In [17]:
qa({'query': 'What is the ontological argument for the existence of God?'})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': 'What is the ontological argument for the existence of God?',
 'result': ' The ontological argument for the existence of God is an argument that moves from the concept or idea of God to the existence of God. It comes in two forms: one from the idea of a Perfect Being and the other from the idea of a Necessary Being. The Perfect Being form argues that the mere concept of God as an absolutely perfect being demands that he exist, because existence is a perfection. The Necessary Being form argues that existence is logically necessary to the concept of a Necessary Being, and therefore a Necessary Being (God) necessarily exists.',
 'source_documents': [Document(page_content='ontological argument (though not the first to call it this) was Anselm (1033–1109). In its simplest form it\nargues from the idea of God to the existence  of God. There are two forms of the argument: one from\nthe idea of a Perfect Being and the other from the idea of a Necessary Being.\nThe Perfect Being.  Acc